# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235578157514                   -0.50    8.0
  2   -7.250418541208       -1.83       -1.41    1.0
  3   -7.251317679228       -3.05       -2.19    3.0
  4   -7.251280407009   +   -4.43       -2.27    3.0
  5   -7.251331816220       -4.29       -2.74    2.0
  6   -7.251338172595       -5.20       -3.21    2.0
  7   -7.251338763509       -6.23       -3.93    2.0
  8   -7.251338796172       -7.49       -4.28    3.0
  9   -7.251338798016       -8.73       -4.64    1.0
 10   -7.251338798674       -9.18       -5.29    3.0
 11   -7.251338798700      -10.58       -5.75    2.0
 12   -7.251338798704      -11.33       -6.51    3.0
 13   -7.251338798705      -13.15       -6.94    3.0
 14   -7.251338798705      -13.88       -7.32    3.0
 15   -7.251338798705      -14.35       -7.89    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05761063553650468
[ Info: Arnoldi iteration step 2: normres = 0.638656028485618
[ Info: Arnoldi iteration step 3: normres = 0.8426697083036139
[ Info: Arnoldi iteration step 4: normres = 0.2456080656282754
[ Info: Arnoldi iteration step 5: normres = 0.557533147640403
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.07e-02, 8.19e-02, 5.01e-01, 2.28e-01, 1.45e-02)
[ Info: Arnoldi iteration step 6: normres = 0.27038688832026336
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.86e-03, 1.20e-01, 1.96e-01, 9.51e-02, 8.57e-02)
[ Info: Arnoldi iteration step 7: normres = 0.12384030172054296
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.17e-04, 1.57e-02, 1.99e-02, 8.27e-02, 7.59e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11055694342413096
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.05e-05